In [15]:
import pandas as pd
from feature_fusion import convert_xlsx_to_csv

def merge_data_by(onwhat,name,qtype):
    data1 =  pd.read_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_drug.csv')
    data1 = data1.drop('Index', axis=1)
    data2 = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Yamanishi/Formatted_Drug_Disease_Data.csv')
    merged_data = data1.merge(data2,on=onwhat,how='left')
    merged_data = merged_data.drop('level_1',axis=1)
    merged_data.to_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_{qtype}.csv',index = False)
for name in ['Es','GPCRs','ICs','NRs']:
        convert_xlsx_to_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_drug.xlsx',f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_drug.csv','KEGG Drug ID')
        convert_xlsx_to_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_protein.xlsx',f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_protein.csv','pro_id')
        merge_data_by('KEGG Drug ID',name,'drug_disease')






In [22]:
def generate_nteraction_matrix(name):
    # 生成药物与疾病的相互作用矩阵
    drug_disease_df = pd.read_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_drug_disease.csv')
    interaction_matrix = pd.crosstab(drug_disease_df['KEGG Drug ID'], drug_disease_df['Efficacy and Disease'])
    #interaction_matrix = interaction_matrix.drop(columns=['KEGG Drug ID']) 
    # 将矩阵保存到新的CSV文件中
    interaction_matrix_file = f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_mat_drug_disease.txt'
    interaction_matrix.to_csv(interaction_matrix_file, sep='\t',index=False,header=False)

    # 返回生成的文件路径
    interaction_matrix_file

In [23]:
for name in ['Es','GPCRs','ICs','NRs']:
    generate_nteraction_matrix(name)

In [8]:
import requests
from bs4 import BeautifulSoup

# 目标网页 URL
url = "https://www.genome.jp/kegg/catalog/pathway_dd.html"

# 发起 HTTP 请求获取网页内容
response = requests.get(url)

# 检查请求是否成功
if response.status_code == 200:
    # 解析网页内容
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 查找疾病相关的表格或列表
    # 假设每个疾病链接在 <a> 标签中并且包含在特定的部分内
    disease_list = []
    
    # 在网页中找到所有的链接或疾病名称
    for link in soup.find_all('0'):
        # 检查链接文字是否和疾病相关
        text = link.text.strip()
        print(text)
        if text and "disease" in text.lower():  # 这里假设疾病相关的链接包含 'disease'
            disease_list.append(text)
    
    # 输出整理的疾病列表
    print("整理出的疾病种类:")
    for disease in disease_list:
        print(disease)
else:
    print(f"无法访问该网页，状态码: {response.status_code}")


整理出的疾病种类:


In [38]:
import re




In [ ]:
# Function to extract disease-related information
def extract_disease_info(kegg_data):
    lines = kegg_data.split('\n')
    #print(lines)
    disease_list  = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Yamanishi/diseases_list.txt', header=None, names=["Disease"])
    for disease in disease_list['Disease']:
        disease_section = [line for line in lines if disease in line]
        #print (disease_section)
    return disease_section


In [53]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

class KEGGProteinDiseaseExtractor:
    def __init__(self, protein_ids):
        """
        Initialize the KEGGProteinDiseaseExtractor with a list of protein IDs.
        """
        self.protein_ids = protein_ids
        self.results = []

    def get_disease_pathways(self, protein_id):
        """
        Fetch KEGG disease pathways for a given protein ID.
        """
        url = f"https://www.kegg.jp/kegg-bin/search_brite?option=-a&search_string={protein_id}&prefix=hsa"
        response = requests.get(url)
        
        if response.status_code == 200:
            brite_data = response.text
            # Extract Human Disease data
            human_disease_data = self.extract_human_disease_data(brite_data)
            cleaned_data = self.clean_html(human_disease_data)
            return cleaned_data
        else:
            return f"Error: Could not retrieve data for {protein_id}"

    def extract_human_disease_data(self, brite_data):
        """
        Extract Human Disease information from KEGG BRITE data using regex.
        """
        # Regex to match the Human Diseases section and its sub-hierarchy
        human_disease_pattern = re.compile(r"(09160 Human Diseases.*?)(09180 Brite Hierarchies|$)", re.S)
        
        match = human_disease_pattern.search(brite_data)
        if match:
            return match.group(1).strip()
        return "No Human Disease data found"

    def clean_html(self, data):
        """
        Clean HTML tags and unnecessary characters from the disease data.
        """
        soup = BeautifulSoup(data, "html.parser")
        cleaned_text = soup.get_text()

        # Clean up extra spaces and HTML entities
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        cleaned_text = re.sub(r'&nbsp;', ' ', cleaned_text)
        cleaned_text = cleaned_text.strip()

        # Only keep text before [PATH:*] pattern
        cleaned_text = re.split(r'\[PATH:', cleaned_text)[0]

        # Remove "09160 Human Diseases" and codes, and format the result
        cleaned_text = cleaned_text.replace("09160 Human Diseases", "").strip()
        cleaned_text = re.sub(r'\d{5} ', '', cleaned_text)  # Remove 5-digit codes
        cleaned_text = re.sub(r',+', ',', re.sub(r'\s+,', ',', cleaned_text))  # Clean extra spaces and commas

        return cleaned_text.strip()

    def fetch_all_diseases(self):
        """
        Fetch disease information for all protein IDs in the list and store the results.
        """
        for pid in self.protein_ids:
            pid = pid[4:]
            print(f"Fetching disease information for protein ID: {pid}")
            diseases = self.get_disease_pathways(pid)
            print(f"Disease information for {pid}:")
            print(diseases)
            print("=" * 50)  # Separator for better readability
            self.results.append([pid, diseases])

    def save_to_csv(self, filename='protein_disease_info.csv'):
        """
        Save the results to a CSV file.
        """
        df = pd.DataFrame(self.results, columns=['Protein ID', 'Disease Information'])
        df.to_csv(filename, index=False)
        print(f"Data has been saved to {filename}")
for name in ['Es','GPCRs','ICs','NRs']:
    protein_ids_list = pd.read_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_protein.csv')
    # Example usage:
    protein_ids = protein_ids_list['pro_id'] # Replace with actual protein IDs

    extractor = KEGGProteinDiseaseExtractor(protein_ids)
    extractor.fetch_all_diseases()
    extractor.save_to_csv()



10
Fetching disease information for protein ID: 1


KeyboardInterrupt: 

In [42]:
import requests

# Example function to get KEGG disease pathways
def get_disease_pathways(protein_id):
    url = f"https://www.kegg.jp/kegg-bin/search_brite?option=-a&search_string={protein_id}&prefix=hsa"
    response = requests.get(url)
    
    if response.status_code == 200:
        brite_data = response.text
        print(brite_data)
        human_disease_data = extract_human_disease_data(brite_data)
        return human_disease_data
    else:
        return f"Error: Could not retrieve data for {protein_id}"
# 用正则表达式匹配Human Disease及其子层级
def extract_human_disease_data(brite_data):
    # 匹配 Human Diseases 部分及其子层级
    human_disease_pattern = re.compile(r"(09160 Human Diseases.*?)(\n  09180|$)", re.S)
    
    # 提取 Human Diseases 部分
    match = human_disease_pattern.search(brite_data)
    print(match)
    if match:
        return match.group(1).strip()
    return None

# Replace these IDs with your actual protein IDs
protein_ids = ['160']  # Example IDs

# Fetch and display disease information for each protein
for pid in protein_ids:
    diseases = get_disease_pathways(pid)
    print(f"Disease information for {pid}:")
    print(diseases)





<html><head><title>Search BRITE</title>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<link rel="stylesheet" type="text/css" href="/css/get_htext.css">
</head><body>
<h2>Search BRITE hierarchies</h2><pre>
KEGG Orthology (KO) [BR:<a href="/brite/hsa00001+160">hsa00001</a>]
&nbsp;&nbsp;09140 Cellular Processes
&nbsp;&nbsp;&nbsp;&nbsp;09141 Transport and catabolism
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;04144 Endocytosis [PATH:<a href="/pathway/hsa04144">hsa04144</a>]
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="/entry/hsa:160"><font color=red>160</font></a> AP2A1; adaptor related protein complex 2 subunit alpha 1	<a href="/entry/K11824">K11824</a> AP2A; AP-2 complex subunit alpha
&nbsp;&nbsp;09150 Organismal Systems
&nbsp;&nbsp;&nbsp;&nbsp;09155 Excretory system
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;04961 Endocrine and other factor-regulated calcium reabsorption [PATH:<a href="/pathway/hsa04961">hsa04961</a>]
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<

In [43]:
print(diseases)

09160 Human Diseases
&nbsp;&nbsp;&nbsp;&nbsp;09164 Neurodegenerative disease
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;05016 Huntington disease [PATH:<a href="/pathway/hsa05016">hsa05016</a>]
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="/entry/hsa:160"><font color=red>160</font></a> AP2A1; adaptor related protein complex 2 subunit alpha 1	<a href="/entry/K11824">K11824</a> AP2A; AP-2 complex subunit alpha
&nbsp;&nbsp;09180 Brite Hierarchies
&nbsp;&nbsp;&nbsp;&nbsp;09182 Protein families: genetic information processing
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;04131 Membrane trafficking [BR:<a href="/brite/hsa04131+160">hsa04131</a>]
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="/entry/hsa:160"><font color=red>160</font></a> AP2A1; adaptor related protein complex 2 subunit alpha 1	<a href="/entry/K11824">K11824</a> AP2A; AP-2 complex subunit alpha
&nbsp;&nbsp;&nbsp;&nbsp;09183 Protein families: signaling and cellular processes
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;04147 Exosome

In [28]:
data = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Yamanishi/diseases_list.txt', header=None, names=["Disease"])
print(data.head())

                                   Disease
0                       Pathways in cancer
1  Transcriptional misregulation in cancer
2                      MicroRNAs in cancer
3                  Proteoglycans in cancer
4    Chemical carcinogenesis - DNA adducts


In [46]:
import pandas as pd
data =pd.read_csv('/data/zyf/HyperGCN-DTI/results/Luo_CV_resluts.csv')
print(data.mean())

Accuracy     0.97582
AUROC        0.99760
AUPr         0.99756
Precision    0.96696
recall       0.98544
f1           0.97606
dtype: float64


In [39]:
import pandas as pd

def convert_xlsx_to_csv(excel_file_path,csv_file_path,qtype):
    df = pd.read_excel(excel_file_path, header=None)
    df.columns = ['Index',qtype]
    df.to_csv(csv_file_path, index=False, encoding='utf-8')
for name in ['Es','GPCRs','ICs','NRs']:
    convert_xlsx_to_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_drug.xlsx',f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_drug_smiles.csv','KEGG Drug ID')
    convert_xlsx_to_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_protein.xlsx',f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_protein_seq.csv','pro_id')

In [40]:

def merge_data_by(onwhat,name,qtype):
    data1 =  pd.read_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_{qtype}.csv')
    print(data1.columns)
    data1 = data1.drop('Index', axis=1)
    data2 = pd.read_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{qtype}.csv')
    merged_data = data1.merge(data2,on=onwhat,how='left')
    merged_data.to_csv(f'/data/zyf/HyperGCN-DTI/data/Yamanishi/{name}/{name}_{qtype}.csv',index=False)

In [41]:
for name in ['Es','GPCRs','ICs','NRs']:
    print(name)
    merge_data_by('KEGG Drug ID',name,'drug_smiles')
    merge_data_by('pro_id',name,'protein_seq')

Es
Index(['Index', 'KEGG Drug ID'], dtype='object')
Index(['Index', 'pro_id'], dtype='object')
GPCRs
Index(['Index', 'KEGG Drug ID'], dtype='object')
Index(['Index', 'pro_id'], dtype='object')
ICs
Index(['Index', 'KEGG Drug ID'], dtype='object')
Index(['Index', 'pro_id'], dtype='object')
NRs
Index(['Index', 'KEGG Drug ID'], dtype='object')
Index(['Index', 'pro_id'], dtype='object')


In [1]:
from utils import *
import torch
import pandas as pd
import yaml
import warnings
from Structe_DPP_HyperGraph import HyGraph_Matrix_DPP_Structure
from hypergraph_utils import generate_G_from_H
import torch.nn.functional as F
import wandb
import os
import torch
import warnings
import numpy as np
from model import *
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from hypergraph_utils import generate_G_from_H
from hypergraph_utils import construct_H_with_KNN
from sklearn.metrics import roc_auc_score, f1_score
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity as cos
from Structe_DPP_HyperGraph import HyGraph_Matrix_DPP_Structure

In [2]:
warnings.filterwarnings("ignore")

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
config['device'] = "cuda" if torch.cuda.is_available() else "cpu"
#config['device'] = "cpu"

In [3]:
seed = 47
args = setup(default_configure, seed)
in_size = 512
hidden_size = 256
out_size = 128
dropout = 0.5
lr = 0.0003
weight_decay = 1e-10
epochs = 800
reg_loss_co = 0.0002
fold = 0
dir = "/data/zyf/HyperGCN-DTI/modelSave"

In [6]:

torch.set_default_dtype(torch.float32)

# for name in ["Luo","Es","GPCRs","ICs","NRs", "Zheng"]:
# dataName: Luo Es GPCRs ICs NRs Zheng
node_num, drug_protein, protein_drug, dtidata = load_dataset_from_name("Luo")
#[708, 1512] torch.Size([708, 1512]) torch.Size([1512, 708]) (3846, 3)
drug_disease,protein_disease = load_disease_info('/data/zyf/HyperGCN-DTI/data/Luo/','mat_drug_disease.txt', 'mat_drug_disease.txt',"Luo")

drug_protein_eye = torch.cat((drug_protein, torch.eye(node_num[0])), dim=1)
protein_drug_eye = torch.cat((protein_drug, torch.eye(node_num[1])), dim=1)
#hd = torch.randn((node_num[0], node_num[0]))
#hp = torch.randn((node_num[1], node_num[1]))
#替换为LLM特征
dti_label = torch.tensor(dtidata[:, 2:3]).to(config['device'])
hd = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Luo/drug_smiles.csv')
hp = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Luo/protein_seq.csv')
features_d = torch.tensor(hd.iloc[:,2:].values,dtype=torch.float32).to(config['device'])
features_p = torch.tensor(hp.iloc[:,2:].values,dtype=torch.float32).to(config['device'])
drug_disease_eye = torch.cat((drug_disease, torch.eye(node_num[0])), dim=1)
protein_disease_eye = torch.cat((protein_disease, torch.eye(node_num[0])), dim=1)

HyGraph_Drug = generate_G_from_H(drug_protein_eye).to(config['device'])
HyGraph_protein = generate_G_from_H(protein_drug_eye).to(config['device'])
HyGraph_Drug_disease = generate_G_from_H(drug_disease_eye).to(config['device'])
HyGraph_protein_disease = generate_G_from_H(protein_disease_eye).to(config['device'])


drug_protein = drug_protein.to(config['device'])
protein_drug = protein_drug.to(config['device'])


HyGraph_Structure_DPP = HyGraph_Matrix_DPP_Structure(dtidata, node_num[0], node_num[1])
HyGraph_Structure_DPP = HyGraph_Structure_DPP.to(config['device'])

data = dtidata
label = dti_label


def train(model, optim, train_index, test_index, epoch, fold):
    model.train()
    out, d, p = model(node_num, features_d, features_p, protein_drug, drug_protein, HyGraph_Drug, HyGraph_protein, train_index, data, HyGraph_Structure_DPP)
    print(out.shape,d.shape,p.shape)
    train_acc = (out.argmax(dim=1) == label[train_index].reshape(-1).long()).sum(dtype=float) / torch.tensor(len(train_index), dtype=float)
    task1_roc = get_roc(out, label[train_index])
    reg = get_L2reg(model.parameters())
    loss = F.nll_loss(out, label[train_index].reshape(-1).long()) + reg_loss_co * reg
    print(train_acc,loss)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    te_acc, te_task1_roc1, te_task1_pr, te_task1_f1 = main_test(model, d, p, test_index, epoch, fold)

    return loss.item(), train_acc, task1_roc, te_acc, te_task1_roc1, te_task1_pr, te_task1_f1


def main_test(model, d, p, test_index, epoch, fold):
    model.eval()
    out = model(node_num, features_d, features_p, protein_drug, drug_protein, HyGraph_Drug, HyGraph_protein, test_index, data, HyGraph_Structure_DPP, iftrain=False, d=d, p=p)

    acc1 = (out.argmax(dim=1) == label[test_index].reshape(-1).long()).sum(dtype=float) / torch.tensor(len(test_index), dtype=float)
    task_roc = get_roc(out, label[test_index])
    task_pr = get_pr(out, label[test_index])
    task_f1 = get_f1score(out, label[test_index])
    # if epoch == 799:
    #     f = open(f"{fold}out.txt","w",encoding="utf-8")
    #     for o in  (out.argmax(dim=1) == label[test_index].reshape(-1)):
    #         f.write(f"{o}\n")
    #     f.close()
    return acc1, task_roc, task_pr, task_f1

def main(tr, te, seed):
    all_acc = []
    all_roc = []
    all_pr = []
    all_f1 = []
    for i in range(len(tr)):
        f = open(f"{i}foldtrain.txt", "w", encoding="utf-8")
        train_index = tr[i]
        for train_index_one in train_index:
            f.write(f"{train_index_one}\n")
        test_index = te[i]
        f = open(f"{i}foldtest.txt", "w", encoding="utf-8")
        for train_index_one in test_index:
            f.write(f"{train_index_one}\n")
        #
        # if not os.path.isdir(f"{dir}"):
        #     os.makedirs(f"{dir}")

        model = HyperGCNDTI(
            num_protein=node_num[1],
            num_drug=node_num[0],
            num_hidden1=512,
            num_hidden2=256,
            num_out=128,
        ).to(config['device'])
            

        # model.load_state_dict(torch.load(f"{dir}/net{i}.pth"))
        optim = torch.optim.Adam(lr=lr, weight_decay=weight_decay, params=model.parameters())
        best_acc = 0
        best_pr = 0
        best_roc = 0
        best_f1 = 0
        for epoch in tqdm(range(epochs)):
            loss, train_acc, task1_roc, acc, task1_roc1, task1_pr, task1_f1 = train(model, optim, train_index, test_index, epoch, i)
            if acc > best_acc:
                best_acc = acc
            if task1_pr > best_pr:
                best_pr = task1_pr
            if task1_roc1 > best_roc:
                best_roc = task1_roc1
            if task1_f1 > best_f1:
                best_f1 = task1_f1
        all_acc.append(best_acc)
        all_roc.append(best_roc)
        all_pr.append(best_pr)
        all_f1.append(best_f1)

    print(f'{"Luo"}, aver Acc is:{sum(all_acc) / len(all_acc):.4f},  Aver roc is:{sum(all_roc) / len(all_roc):.4f}, '
    f'Aver Pr is:{sum(all_pr) / len(all_pr):.4f} ,Aver f1 is:{sum(all_f1) / len(all_f1):.4f}')


train_indeces, test_indeces = get_cross(dtidata)
main(train_indeces, test_indeces, config['seed'])


  0%|          | 0/800 [00:00<?, ?it/s]

1 torch.Size([2220, 128])
torch.Size([2220, 2220])
3 torch.Size([2220, 128])
torch.Size([708, 128]) torch.Size([1512, 128])
torch.Size([3846, 256])


  0%|          | 0/800 [00:00<?, ?it/s]

torch.Size([3076, 2]) torch.Size([708, 128]) torch.Size([1512, 128])
tensor(0.4880, device='cuda:0', dtype=torch.float64) tensor(0.8081, device='cuda:0', grad_fn=<AddBackward0>)


RuntimeError: Function AddmmBackward returned an invalid gradient at index 2 - got [320, 512] but expected shape compatible with [1280, 512]

In [8]:


# Check if model is on CUDA or CPU
print("Model is on device:", next(model.parameters()).device)

# Example tensors
features_d = features_d.to(config['device'])
features_p = features_p.to(config['device'])

# Check if tensors are on CUDA or CPU
print("features_d is on device:", features_d.device)
print("features_p is on device:", features_p.device)


NameError: name 'model' is not defined

In [5]:
drug_diease_eye = torch.cat((drug_disease, torch.eye(node_num[0])), dim=1)
protein_disease_eye = torch.cat((protein_disease, torch.eye(node_num[0])), dim=1)

HyGraph_Drug = generate_G_from_H(drug_protein_eye).to(config['device'])
HyGraph_protein = generate_G_from_H(protein_drug_eye).to(config['device'])
HyGraph_Drug_disease = generate_G_from_H(drug_diease_eye).to(config['device'])
HyGraph_protein_disease = generate_G_from_H(protein_disease_eye).to(config['device'])


drug_protein = drug_protein.to(config['device'])
protein_drug = protein_drug.to(config['device'])


HyGraph_Structure_DPP = HyGraph_Matrix_DPP_Structure(dtidata, node_num[0], node_num[1])
HyGraph_Structure_DPP = HyGraph_Structure_DPP.to(config['device'])


In [10]:
import math
import torch
from layer import *
import numpy as np
import torch.nn as nn
from loaddata_utils import *
import torch.nn.functional as F
from torch.autograd import Variable

In [13]:
from model import *

In [14]:
class HyperGCNDTI(nn.Module):
    def __init__(self, num_protein, num_drug, num_hidden1, num_hidden2, num_out, dropout=0.5):  # 1512, 708, 512, 128
        super(HyperGCNDTI, self).__init__()
        self.DPP_GCN = DPP_GCN(num_hidden2, dropout)
        self.MLP = MLP(num_hidden2)
        self.alpha = 0.8

        self.linear_d1 = nn.Linear(num_drug, num_hidden1)
        self.linear_d2 = nn.Linear(num_hidden1, num_hidden2)
        self.linear_d3 = nn.Linear(num_hidden2, num_out)

        self.linear_p1 = nn.Linear(num_protein, num_hidden1)
        self.linear_p2 = nn.Linear(num_hidden1, num_hidden2)
        self.linear_p3 = nn.Linear(num_hidden2, num_out)

        self.c = nn.Parameter(torch.Tensor((3), 1, 1))
        self.d = nn.Parameter(torch.Tensor((3), 1, 1))

        nn.init.constant_(self.c, 1)
        nn.init.constant_(self.d, 1)

    def SGC(self, feature, adj):

        adj = adj + (torch.eye(adj.shape[0]).cuda()) * 2
        deg = torch.sum(adj, dim=1)
        deg[deg <= 1e-10] = 1
        deg_inv = deg.pow(-0.5)
        deg_inv = deg_inv * torch.eye(adj.shape[0]).type(torch.FloatTensor).cuda()
        adj = torch.mm(deg_inv, adj)
        adj = torch.mm(adj, deg_inv).type(torch.FloatTensor)

        output = torch.mm(adj.cuda(), feature.cuda())

        return output

    def forward(self, node_num, drug_vec, protein_vec, protein_drug, drug_protein, HyGraph_Drug, HyGraph_protein, dateset_index, data, HyGraph_Structure_DPP, iftrain=True, d=None, p=None):

        drug1 = F.relu(self.linear_d1(drug_vec))
        drug2 = F.relu(self.linear_d2(drug1))
        drug3 = F.relu(self.linear_d3(drug2))

        protein1 = F.relu(self.linear_p1(protein_vec))
        protein2 = F.relu(self.linear_p2(protein1))
        protein3 = F.relu(self.linear_p3(protein2))
        feature = torch.cat((drug3, protein3), dim=0)

        HyGraph_ToGraph = torch.zeros((node_num[0] + node_num[1], node_num[0] + node_num[1])).cuda()
        HyGraph_ToGraph[:node_num[0], node_num[0]:] = drug_protein
        HyGraph_ToGraph[node_num[0]:, :node_num[0]] = protein_drug
        HyGraph_ToGraph[:node_num[0], :node_num[0]] = HyGraph_Drug
        HyGraph_ToGraph[node_num[0]:, node_num[0]:] = HyGraph_protein


        adj = HyGraph_ToGraph
        X_conv1 = self.SGC(feature, adj)
        X_conv2 = self.SGC(X_conv1, adj)
        X_conv3 = self.SGC(X_conv2, adj)
        X_conv4 = self.SGC(X_conv3, adj)
        X_conv5 = self.SGC(X_conv4, adj)
        conv_sum = self.SGC(X_conv5, adj)
        conv_sumx = self.SGC(conv_sum, adj)
        drug_feature_ht = conv_sumx[:drug_vec.shape[0]]
        protein_feature_ht = conv_sumx[drug_vec.shape[0]:]

        if iftrain:
            d, p = drug_feature_ht, protein_feature_ht

        feature_dp = torch.cat((d[data[:, :1]], p[data[:, 1:2]]), dim=2)

        feature_dp = feature_dp.squeeze(1)

        f_edge, f_feature = constructure_knngraph(data, d, p)

        z1, z2, z3, z4, z5, z6 = self.DPP_GCN(feature_dp, HyGraph_Structure_DPP, f_feature, f_edge)

        att3 = F.softmax(self.c, dim=0)
        feature_hg = torch.stack((z1, z2, z3), dim=0)
        feature_hg = torch.sum((att3 * feature_hg), dim=0)

        att4 = F.softmax(self.d, dim=0)
        feature_knn = torch.stack((z4, z5, z6), dim=0)
        feature_knn = torch.sum((att4 * feature_knn), dim=0)

        feature_stack = torch.cat((feature_hg, feature_knn), dim=1)
        pred = self.MLP(feature_stack[dateset_index])

        if iftrain:
            return pred, d, p
        return pred

In [15]:
model =HyperGCNDTI(
            num_protein=node_num[1],
            num_drug=node_num[0],
            num_hidden1=512,
            num_hidden2=256,
            num_out=128,
        ).to(config['device'])

In [21]:
protein_drug.shape

torch.Size([1512, 708])

In [20]:
HyGraph_Drug.shape

torch.Size([708, 708])

In [22]:
HyGraph_ToGraph = torch.zeros((node_num[0] + node_num[1], node_num[0] + node_num[1])).cuda()
HyGraph_ToGraph[:node_num[0], node_num[0]:] = drug_protein
HyGraph_ToGraph[node_num[0]:, :node_num[0]] = protein_drug
HyGraph_ToGraph[:node_num[0], :node_num[0]] = HyGraph_Drug
HyGraph_ToGraph[node_num[0]:, node_num[0]:] = HyGraph_protein

In [23]:
HyGraph_ToGraph.shape

torch.Size([2220, 2220])

In [25]:
HyGraph_Structure_DPP.shape

torch.Size([3846, 3846])

In [ ]:
# Forward pass
out, d, p = model(
    node_num, 
    features_d, #drug_vec
    features_p, #provec
    protein_drug, 
    drug_protein, 
    HyGraph_Drug, 
    HyGraph_protein, 
    train_index, 
    dtidata, 
    HyGraph_Structure_DPP
)

In [6]:

def compute_accuracy(predictions, labels):
    """Compute accuracy based on predictions and true labels."""
    return (predictions.argmax(dim=1) == labels.reshape(-1).long()).sum(dtype=float) / torch.tensor(len(labels), dtype=float)

def compute_loss(output, target, model, reg_loss_co):
    """Calculate the loss including L2 regularization."""
    classification_loss = F.nll_loss(output, target.reshape(-1).long())
    reg = get_L2reg(model.parameters())
    total_loss = classification_loss + reg_loss_co * reg
    return total_loss

def train(model, optim, train_index, test_index, epoch, fold, config):
    """
    Train the model for one epoch.

    Args:
        model: The PyTorch model to be trained.
        optim: The optimizer used for training.
        train_index: Indices for training samples.
        test_index: Indices for testing samples.
        epoch: Current epoch number.
        fold: Current fold number for cross-validation.
        config: A dictionary containing configuration parameters.

    Returns:
        Tuple containing training loss, accuracy, ROC, and test metrics.
    """
    
    model =HyperGCNDTI(
                num_protein=node_num[1],
                num_drug=node_num[0],
                num_hidden1=512,
                num_hidden2=256,
                num_out=128,
            ).to(config['device'])
    # Set model to training mode
    model.train()

    # Forward pass
    out, d, p = model(
        node_num, 
        features_d, 
        features_p, 
        protein_drug, 
        drug_protein, 
        HyGraph_Drug, 
        HyGraph_protein, 
        train_index, 
        dtidata, 
        HyGraph_Structure_DPP
    )

    # Compute training accuracy
    train_acc = compute_accuracy(out, config['label'][train_index])
    
    # Compute ROC
    task1_roc = get_roc(out, config['label'][train_index])
    
    # Compute loss
    loss = compute_loss(out, config['label'][train_index], model, config['reg_loss_co'])

    # Backpropagation and optimization step
    optim.zero_grad()
    loss.backward()
    optim.step()

    # Perform testing
    te_acc, te_task1_roc1, te_task1_pr, te_task1_f1 = main_test(model, d, p, test_index, epoch, fold)

    # Log metrics to W&B
    wandb.log({
        "epoch": epoch,
        "fold": fold,
        "train_loss": loss.item(),
        "train_accuracy": train_acc,
        "train_roc": task1_roc,
        "test_accuracy": te_acc,
        "test_roc": te_task1_roc1,
        "test_precision": te_task1_pr,
        "test_f1": te_task1_f1
    })

    # Return the results
    return loss.item(), train_acc, task1_roc, te_acc, te_task1_roc1, te_task1_pr, te_task1_f1
